In [ ]:
import re
import subprocess

from multiprocessing import Process, JoinableQueue as Queue
from os import listdir, makedirs, path
from queue import Empty as QueueEmptyException

from metadata.cameras import INPUT_DIRS

INPUT_PATH = "/home/vmubuntu/OitoDeJaneiro/vids/FULL-0801"
OUTPUT_PATH = "/home/vmubuntu/OitoDeJaneiro/vids/0801-500"

FFMPEG_CMD = 'ffmpeg -i "INPUT_PATH" -vf scale=500:-2 -an "OUTPUT_PATH"'

In [ ]:
def run_ffmpeg(q):
  while True:
    try:
      ip, op = q.get(timeout=3)
    except QueueEmptyException:
      return
    my_cmd = FFMPEG_CMD.replace("INPUT_PATH", ip).replace("OUTPUT_PATH", op)
    print(my_cmd)
    ret = subprocess.run(my_cmd, shell=True)
    q.task_done()

In [ ]:
to_process = Queue()

for io_dir in INPUT_DIRS:
  input_dir_path = path.join(INPUT_PATH, io_dir)
  output_dir_path = path.join(OUTPUT_PATH, io_dir)
  makedirs(output_dir_path, exist_ok=True)

  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])
  for io_file in input_files:
    output_file_path = path.join(output_dir_path, io_file)
    if not path.exists(output_file_path):
      input_file_path = path.join(input_dir_path, io_file)
      to_process.put((input_file_path, output_file_path))

In [ ]:
for _ in range(4):
  Process(target=run_ffmpeg, args=(to_process,)).start()

to_process.join()
print("DONE")